<a href="https://colab.research.google.com/github/Itsuki-Hamano123/auto_ml/blob/master/tool_comparison/adult_census/tpot_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q tpot
%pip show tpot scikit-learn

     |████████████████████████████████| 92kB 4.0MB/s 
     |████████████████████████████████| 163kB 8.8MB/s 
Name: TPOT
Version: 0.11.6.post1
Summary: Tree-based Pipeline Optimization Tool
Home-page: https://github.com/EpistasisLab/tpot
Author: Randal S. Olson
Author-email: rso@randalolson.com
License: GNU/LGPLv3
Location: /usr/local/lib/python3.6/dist-packages
Requires: numpy, tqdm, update-checker, deap, joblib, pandas, scikit-learn, stopit, scipy
Required-by: 
---
Name: scikit-learn
Version: 0.22.2.post1
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: None
Author-email: None
License: new BSD
Location: /usr/local/lib/python3.6/dist-packages
Requires: numpy, scipy, joblib
Required-by: yellowbrick, umap-learn, TPOT, textgenrnn, sklearn, sklearn-pandas, mlxtend, lucid, lightgbm, librosa, imbalanced-learn, fancyimpute


In [2]:
import cloudpickle, datetime, os

import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tpot import TPOTClassifier
import xgboost

## データセット読み込み
- [国税調査データセット:https://www.openml.org/d/1590](https://www.openml.org/d/1590)

In [3]:
%%time
DATA_ID = '1590' #@param{type:'string'}

# データセットをフェッチ
result = fetch_openml(data_id=DATA_ID,
                    as_frame=True)
X = result.data
y = result.target
x_cols = result.feature_names
y_col = result.target_names

display(x_cols)
display(X.head())
display(y_col)
display(y.head())


def proc_y(y):
    '''ラベルの加工'''

    def _laebl_encode(y):
        '''ラベルエンコードしたnumpy.ndarrayを返す'''
        le = LabelEncoder()
        return le.fit_transform(y)

    proc_y = _laebl_encode(y=y)
    return proc_y


def proc_X(X, one_hot_encode_cols):
    '''特徴量の加工'''

    def _one_hot_encode(X, X_cols, drop_first=True):
        '''ワンホットエンコードしたDataframeを返す'''
        return pd.get_dummies(X, columns=X_cols,
                              drop_first=drop_first)

    proc_X = _one_hot_encode(X=X, X_cols=one_hot_encode_cols)
    return proc_X


one_hot_encode_cols = ['workclass', 'education', 'marital-status', 'occupation',
                       'relationship', 'race', 'sex', 'native-country']
X=proc_X(X=X, one_hot_encode_cols=one_hot_encode_cols)
y=proc_y(y=y)

display(X.head())
display(y)


SEED = 7 #@param{type:'number'}
SPLIT = 0.2 #@param{type:'number'}

train_X, test_X, train_y, test_y = train_test_split(X, y,
                                                    test_size=SPLIT,
                                                    random_state=SEED,
                                                    shuffle=True)

display('train shape:{}'.format(train_X.shape))
display('test shape:{}'.format(test_X.shape))

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country']

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States
3,44.0,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States
4,18.0,NaN,103497.0,Some-college,10.0,Never-married,NaN,Own-child,White,Female,0.0,0.0,30.0,United-States


['class']

0    <=50K
1    <=50K
2     >50K
3     >50K
4    <=50K
Name: class, dtype: category
Categories (2, object): ['>50K', '<=50K']

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Self-emp-not-inc,workclass_Self-emp-inc,workclass_Federal-gov,workclass_Local-gov,workclass_State-gov,workclass_Without-pay,workclass_Never-worked,education_Some-college,education_11th,education_HS-grad,education_Prof-school,education_Assoc-acdm,education_Assoc-voc,education_9th,education_7th-8th,education_12th,education_Masters,education_1st-4th,education_10th,education_Doctorate,education_5th-6th,education_Preschool,marital-status_Divorced,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,marital-status_Married-spouse-absent,marital-status_Married-AF-spouse,occupation_Craft-repair,occupation_Other-service,occupation_Sales,occupation_Exec-managerial,occupation_Prof-specialty,occupation_Handlers-cleaners,...,native-country_Cambodia,native-country_England,native-country_Puerto-Rico,native-country_Canada,native-country_Germany,native-country_Outlying-US(Guam-USVI-etc),native-country_India,native-country_Japan,native-country_Greece,native-country_South,native-country_China,native-country_Cuba,native-country_Iran,native-country_Honduras,native-country_Philippines,native-country_Italy,native-country_Poland,native-country_Jamaica,native-country_Vietnam,native-country_Mexico,native-country_Portugal,native-country_Ireland,native-country_France,native-country_Dominican-Republic,native-country_Laos,native-country_Ecuador,native-country_Taiwan,native-country_Haiti,native-country_Columbia,native-country_Hungary,native-country_Guatemala,native-country_Nicaragua,native-country_Scotland,native-country_Thailand,native-country_Yugoslavia,native-country_El-Salvador,native-country_Trinadad&Tobago,native-country_Peru,native-country_Hong,native-country_Holand-Netherlands
0,25.0,226802.0,7.0,0.0,0.0,40.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,38.0,89814.0,9.0,0.0,0.0,50.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,28.0,336951.0,12.0,0.0,0.0,40.0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,44.0,160323.0,10.0,7688.0,0.0,40.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,18.0,103497.0,10.0,0.0,0.0,30.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


array([0, 0, 1, ..., 0, 0, 1])

'train shape:(39073, 97)'

'test shape:(9769, 97)'

CPU times: user 1.31 s, sys: 61.3 ms, total: 1.37 s
Wall time: 7.9 s


# パイプライン構築

In [4]:
GENERATION =  100#@param{type:'number'}
POPULATION =  100#@param{type:'number'}
CV =  5#@param{type:'number'}
EARLY_STOP_ROUND = 2 #@param:{type:'number'}
N_JOBS = -1 #@param{type:'number'}
VERBOSITY = 3 #@param{type:'number'}

SEARCH_STRATEGY = 'TPOT sparse'

tpot = TPOTClassifier(config_dict=SEARCH_STRATEGY,
                      generations=GENERATION, # (default:100)
                      population_size=POPULATION, # (default:100)
                      cv=CV, #(default:5)
                      early_stop=EARLY_STOP_ROUND,
                      n_jobs=N_JOBS, # (default:1)
                      random_state=SEED,
                      verbosity=VERBOSITY,
                      warm_start=True, # 以前のfit結果があれば続きから開始
                      )


In [5]:
%%time
tpot.fit(features=train_X, target=train_y)

13 operators have been imported by TPOT.


Skipped pipeline #24 due to time out. Continuing to the next pipeline.
_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only dual=False, got dual=True.
_pre_test decorator: _random_mutation_operator: num_test=1 Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty..
_pre_test decorator: _random_mutation_operator: num_test=2 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required..
_pre_test decorator: _random_mutation_operator: num_test=0 Expected n_neighbors <= n_samples,  but n_samples = 50, n_neighbors = 54.
_pre_test decorator: _random_mutation_operator: num_test=0 feature_names mismatch: ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'workclass_Self-emp-not-inc', 'workclass_Self-emp-inc', 'workclass_Federal-gov', 'workclass_Local-gov', 'workclass_State-gov', 'workclass_Without-pay', 'workclass_Never-worked', 'education_Some-college', 'education_11th', 'education_HS-grad', 'ed

TPOTClassifier(config_dict='TPOT sparse', crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=2, generations=100,
               log_file=None, max_eval_time_mins=5, max_time_mins=None,
               memory=None, mutation_rate=0.9, n_jobs=-1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=100,
               random_state=7, scoring=None, subsample=1.0, template=None,
               use_dask=False, verbosity=3, warm_start=True)

In [6]:
# 探索されたパイプライン表示
#display(tpot.pareto_front_fitted_pipelines_)

# ベストなパイプライン表示
display(tpot.fitted_pipeline_)

Pipeline(memory=None,
         steps=[('selectpercentile',
                 SelectPercentile(percentile=56,
                                  score_func=<function f_classif at 0x7f179e61b8c8>)),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=7,
                               min_child_weight=4, missing=None,
                               n_estimators=100, n_jobs=1, nthread=1,
                               objective='binary:logistic', random_state=7,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None,
                               subsample=0.9500000000000001, verbosity=1))],
         verbose=False)

#パイプラインの出力

In [7]:
%cd ./drive/My Drive/機械学習練習/AutoML/ツールの比較/adult_census
%ls

/content/drive/My Drive/機械学習練習/AutoML/ツールの比較/adult_census
autokeras_classifier.ipynb                 Prototype_Model/
export_pipeline/                           tpot_classifier.ipynb
kerasTuner_one_hot_DNN_classifier.ipynb    xgb_classifier.ipynb
kerasTuner_WideDeepModel_classifier.ipynb


In [8]:
%%time

EXPORT_DIR =  '../export_pipeline' # @param {type:'string'}
file_prefix = 'adult_census_classification_pipeline' # @param{type:'string'}

now = datetime.datetime.now().strftime('%Y%m%d_%H%m%s')
export_file = os.path.join(EXPORT_DIR, now+'_'+file_prefix)

# ベストなパイプラインのスクリプトファイル出力
tpot.export(output_file_name=export_file+'.py')


def dump_pkl(obj, path):
    '''objをpklファイルで出力'''
    with open(path, 'wb') as f:
        f.write(cloudpickle.dumps(obj))

# ベストなパイプラインをpklで出力
best_pipline = tpot.fitted_pipeline_
dump_pkl(best_pipline, export_file+'.pkl')

%ls {EXPORT_DIR}

20201106_04111604636365_radon-regressor_pipeline.pkl
20201106_04111604636365_radon-regressor_pipeline.py
20201106_04111604636433_radon-regressor_pipeline.pkl
20201106_04111604636433_radon-regressor_pipeline.py
20201113_02111605233126_adult_census_classification_pipeline.pkl
20201113_02111605233126_adult_census_classification_pipeline.py
20201128_07111606548988_adult_census_classification_pipeline.pkl
20201128_07111606548988_adult_census_classification_pipeline.py
CPU times: user 9.25 ms, sys: 8.11 ms, total: 17.4 ms
Wall time: 528 ms


# モデルの評価

## TPOTの組み込み評価関数

In [9]:
%%time
score = tpot.score(testing_features=test_X, testing_target=test_y)
display(score)

0.8693827413245983

CPU times: user 82 ms, sys: 6 ms, total: 88 ms
Wall time: 89.1 ms


## sklearnの評価関数

In [11]:
%%time
# 予測結果の取得
train_y_pred = tpot.predict(features=train_X)
test_y_pred = tpot.predict(features=test_X)

train_report = classification_report(y_true=train_y, y_pred=train_y_pred)
test_report = classification_report(y_true=test_y, y_pred=test_y_pred)

print(train_report)
print('----- -----')
print(test_report)



              precision    recall  f1-score   support

           0       0.90      0.95      0.92     29696
           1       0.81      0.65      0.72      9377

    accuracy                           0.88     39073
   macro avg       0.85      0.80      0.82     39073
weighted avg       0.88      0.88      0.88     39073

----- -----
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      7459
           1       0.78      0.62      0.69      2310

    accuracy                           0.87      9769
   macro avg       0.84      0.78      0.80      9769
weighted avg       0.86      0.87      0.86      9769

CPU times: user 469 ms, sys: 5.92 ms, total: 474 ms
Wall time: 475 ms
